In [ ]:
import pylabnet.hardware.awg.zi_hdawg as zi_hdawg
from pylabnet.utils.logging.logger import LogClient

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import time

from IPython.display import clear_output, display

import pyvisa
from pylabnet.utils.logging.logger import LogClient
from pylabnet.network.client_server import agilent_e4405B
import pylabnet.hardware.spectrum_analyzer.agilent_e4405B as sa_hardware


import itertools as it
import pandas as pd
import seaborn as sns

from pylabnet.utils.logging.logger import LogService
from pylabnet.network.core.generic_server import GenericServer
import os
import sys
from pylabnet.utils.optimizer import IQOptimizer
from pylabnet.network.client_server import HMC_T2220

# Instantiante HDAWG

In [ ]:
dev_id = 'dev8227'

# Instantiate
logger = LogClient(
    host='localhost',
    port=12351,
    module_tag=f'ZI HDAWG {dev_id}'
)

In [ ]:
# Instantiate Hardware class
hd = zi_hdawg.Driver(dev_id, logger)

# Select channel grouping
hd.set_channel_grouping(0)

hd.enable_output(0)
hd.enable_output(1)

# MW client

In [ ]:
# Connect to MW source client
mw_client = HMC_T2220.Client(
    host='localhost',
    port=12378
)

In [ ]:
#mw_client.output_on()
pow = 20  # -20 dBm
mw_client.set_power(pow)
mw_client.is_output_on()
mw_client.get_power()

# Connect to Spectrum Analyzer

In [ ]:
sa = agilent_e4405B.Client(
    host='localhost',
    port=12356
)

# Reset spectrum analzyer
sa.write('*RST')

# Optimize

In [ ]:
opt1 = IQOptimizer(mw_client, hd, sa, 11.4e9, 300e6, max_lower_sideband_pow = -55, max_carrier_pow = -55, param_guess = ([70, 0.7, 0.65, 0.01, 0.01]), phase_window = 22, q_window = 0.4, dc_i_window = 0.1, dc_q_window = 0.1)
#opt1.opt()

In [ ]:
#opt1.sa.plot_trace()
#opt1.opt_lower_sideband()
#opt1.opt_carrier()
opt1.opt()

In [ ]:
#opt1.opt_carrier()

In [ ]:
#opt1.opt_lower_sideband()
# opt1.initialize_reopt_params()
# opt1.opt()

In [ ]:
#opt1.opt_phase

In [ ]:
#opt1.opt_q